## General Info

### classes
769 Cue onset left (class 1)  <br>
770 Cue onset right (class 2)  <br>
771 Cue onset foot (class 3) <br>
772 Cue onset tongue (class 4)  <br>

### Data
Input data is generally a 4D tensor with size (9, 288, 22, 1000)

9 subjects <br>
288 trials for each subject <br>
25 channels (22 EEGs and 3 EOGs, only EEGs matter) <br>
1000 time steps (4 seconds at 250 Hz) <br>

Output data is generally a 2D matrix with size (9, 288)

Each session consists of 288 trials. Each session is comprised of 6 runs separated by short breaks. One run consists of 48 trials (12 for each of the four possible classes), yielding a total of 288 trials per session. 

1 (session) = 288 trials = 4 (classes) * 12 (class repetitions) * 6 (runs)

Two sessions on different days were recorded for each subject?

288 trials of (4 seconds of active imagination + 1.5 seconds rest) for each session.

# Data Preprocessing

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Softmax
from keras.layers import LSTM, GRU, advanced_activations
from keras import optimizers
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, Conv2D
from keras import backend as K

In [6]:
# if true, we train one individual subjects, if False, we train across all subjects combined.
Process_data_each_subject = False

if Process_data_each_subject == True:
    Subject_train = 1   # which subject to train on

# number of chuncks that we break the signal into. For example, if Time_devide = 10, we break each signal into 10 signals
# with 100 time points
Time_devide = 10

# Number of trials used for testing
Num_test=50

Architecture = 4    # {1,2,3,4,5,6}

epochs = 1             # Number of epochs
N_mont = 20        # Number of iterations

batch_num = 10      # number of batches (as opposed to batch size)

# Run the code!

In [7]:
# Make Sure You are adding your directory 
A01T = h5py.File('project_datasets\A01T_slice.mat','r')
#A01T = h5py.File('C:\\Users\\manit\\Downloads\\Projects\\project_datasets\\A01T_slice.mat','r')
#A01T = h5py.File('A01T_slice.mat','r')
X = np.copy(A01T['image'])
y = np.copy(A01T['type'])
y = y[0,0:X.shape[0]:1]
y = np.asarray(y, dtype = np.int32)

OSError: Unable to open file (unable to open file: name = 'project_datasets\A01T_slice.mat', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# Data processing (creating training, validation, and test sets)

# data is saved in the project_datasets folder
# project_datasets folder is placed in the same folder as this notebook

num_class=4;
num_subject=9;
Data={}

for i in range(num_subject):
    address = 'project_datasets\\' + 'A0' + str(i+1) + 'T_slice.mat' # address where data is saved
    #address='C:\\Users\\manit\\Downloads\\Projects\\project_datasets\\'+'A0' + str(i+1) + 'T_slice.mat'
    #address ='A0' + str(i+1) + 'T_slice.mat'
    A01T = {}
    A01T = h5py.File(address, 'r')
    Data['X'+str(i+1)] = np.copy(A01T['image'])
    y = np.copy(A01T['type'])
    y = y[0,0:Data['X'+str(i+1)].shape[0]:1]
    Data['y'+str(i+1)] = np.asarray(y, dtype=np.int32)

In [ ]:
# deleting nans

for i in range(num_subject):
    Temp1=Data['X'+str(i+1)]
    Temp2=Data['y'+str(i+1)]
    Index_drop=np.argwhere(np.isnan(Temp1))[:,0]
    Index=list(set(Index_drop))
    Data['X'+str(i+1)]=np.delete(Temp1,Index,axis=0)
    Data['y'+str(i+1)]=np.delete(Temp2,Index)

In [ ]:
# creating training, validation, and test sets and removing EOG channels
# updating labels to {0,1,2,3}

Train_X, Test_X = {},{}
Train_y, Test_y = {},{}
Data_X, Data_y = {},{}


for i in range(num_subject):
    
    # getting trial indexes and shuffle them
    index=np.arange(Data['y'+str(i+1)].shape[0])
    np.random.shuffle(index)
    
    # generating training, validation, and test sets
    test_index=index[0:Num_test]
    train_index=index[Num_test:]   
    
    Data_X[i+1] = (Data['X'+str(i+1)][index,0:22])
    Data_y[i+1] = Data['y'+str(i+1)] - 769
    
    Train_X[i+1]=(Data['X'+str(i+1)][train_index,0:22])
    Train_y[i+1]=Data['y'+str(i+1)][train_index] - 769
    
    Test_X[i+1]=(Data['X'+str(i+1)][test_index,0:22])
    Test_y[i+1]=Data['y'+str(i+1)][test_index] - 769

    
print(Data_X[4].shape)
print(Train_X[4].shape)
print(Test_X[4].shape)

In [ ]:
# remoing mean from data for each subject

if (Process_data_each_subject):
    Mean_sub = {}     #  22 * 1000

    for i in np.arange(num_subject):

        Mean_sub[i+1] = np.mean(Data_X[i+1],axis = 0)   # mean across trials
        Train_X[i+1] -= Mean_sub[i+1]                   # broadcating
        Test_X[i+1] -= Mean_sub[i+1]

    print(Mean_sub[4].shape)
    print(Train_X[4].shape)

In [ ]:
# breaking data across subjects
# changing from ? * 1000 * 22 to (?*9) * 1000 * 22

if (Process_data_each_subject==False):
    
    X_train_all = Train_X[1]
    X_test_all = Test_X[1]
    X_Data_all = Data_X[1]

    y_train_all = Train_y[1]
    y_test_all = Test_y[1]
    y_Data_all = Data_y[1]

    for subject in np.arange(2,num_subject+1):

        X_Data_all = np.concatenate((X_Data_all,Data_X[subject]),axis = 0)
        X_train_all = np.concatenate((X_train_all,Train_X[subject]),axis = 0)
        X_test_all = np.concatenate((X_test_all,Test_X[subject]),axis = 0)

        y_Data_all = np.concatenate((y_Data_all,Data_y[subject]),axis = 0)
        y_train_all = np.concatenate((y_train_all,Train_y[subject]),axis = 0)
        y_test_all = np.concatenate((y_test_all,Test_y[subject]),axis = 0)


    print(X_Data_all.shape)
    print(X_train_all.shape)
    print(X_test_all.shape)

In [ ]:
# remoing mean from data for all

if (Process_data_each_subject==False):
    Mean_all = {}     #  22 * 1000

    Mean_all = np.mean(X_Data_all,axis = 0)   # mean across trials

    X_Data_all -= Mean_all
    X_train_all -= Mean_all                   # broadcating
    X_test_all -= Mean_all

    print(Mean_all.shape)
    print(X_Data_all.shape)

In [ ]:
if (Process_data_each_subject == False):

    X_train_all = np.swapaxes(X_train_all,0,1)
    X_train_all = np.reshape(X_train_all,(X_train_all.shape[0],X_train_all.shape[1]*Time_devide,X_train_all.shape[2]//Time_devide))
    X_train_all = np.swapaxes(X_train_all,0,1)

    X_test_all = np.swapaxes(X_test_all,0,1)
    X_test_all = np.reshape(X_test_all,(X_test_all.shape[0],X_test_all.shape[1]*Time_devide,X_test_all.shape[2]//Time_devide))
    X_test_all = np.swapaxes(X_test_all,0,1)

    
if (Process_data_each_subject == True):
    
    for i in np.arange(0,num_subject):
        
        Train_X[i+1] = np.swapaxes(Train_X[i+1],0,1)
        Train_X[i+1] = np.reshape(Train_X[i+1],(Train_X[i+1].shape[0],Train_X[i+1].shape[1]*Time_devide,Train_X[i+1].shape[2]//Time_devide))
        Train_X[i+1] = np.swapaxes(Train_X[i+1],0,1)
        
        Test_X[i+1] = np.swapaxes(Test_X[i+1],0,1)
        Test_X[i+1] = np.reshape(Test_X[i+1],(Test_X[i+1].shape[0],Test_X[i+1].shape[1]*Time_devide,Test_X[i+1].shape[2]//Time_devide))
        Test_X[i+1] = np.swapaxes(Test_X[i+1],0,1)

In [ ]:
if (Process_data_each_subject == False):
    y_train_all = np.repeat(y_train_all,Time_devide)
    y_test_all = np.repeat(y_test_all,Time_devide)
    
    X_train_all = np.swapaxes(X_train_all,1,2)
    X_test_all = np.swapaxes(X_test_all,1,2)
    
if (Process_data_each_subject == True):
    for i in np.arange(0,num_subject):
        
        Train_y[i+1] = np.repeat(Train_y[i+1],Time_devide)
        Test_y[i+1] = np.repeat(Test_y[i+1],Time_devide)
    
        Test_X[i+1] = np.swapaxes(Test_X[i+1],1,2)
        Train_X[i+1] = np.swapaxes(Train_X[i+1],1,2)

# Project with Keras

# Code for training all / testing all

if the the layer before last layer is Conv, flatten needs to be used.

In [ ]:
if (Process_data_each_subject == False):
    K.clear_session()    # clearing previous session

    num_classes = 4
    signal_length = 1000//Time_devide
    number_channel = 22

    input_shape = (signal_length,number_channel)  
   
    batch_size = X_train_all.shape[0]//batch_num

    print('Loading data...')

    x_train = X_train_all             # ? * 1000 * 22
    y_train = y_train_all             # ?

    x_test = X_test_all                 # ? * 1000 * 22
    y_test = y_test_all                 # ?

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    # # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    print(x_train.shape, 'train_x samples')
    print(y_train.shape, 'train_y samples')
    print(x_test.shape, 'test samples')

    print('Build model...')

    model = Sequential()
    
    if (Architecture == 1):

        # Conv
        model.add(Conv1D(32, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.6))

        # LSTM
        model.add(LSTM(128, return_sequences=False))
        model.add(Dropout(0.5))
        
    if (Architecture == 2):
        
        model.add(Conv1D(32, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.5))
        
        model.add(Conv1D(32, kernel_size = 11))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.5))
        
        model.add(Conv1D(32, kernel_size = 11))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.5))
        
        model.add(Flatten())
        # FC
        model.add(Dense(64,activation = 'relu',input_shape=input_shape))
    
    if (Architecture == 3):

        # Conv
        model.add(Conv1D(64, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.6))
        
        model.add(Conv1D(64, kernel_size = 11))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.6))

        model.add(Flatten())
        # FC
        model.add(Dense(64,activation = 'relu',input_shape=input_shape))
        
    if (Architecture == 4):

        # Conv
        model.add(Conv1D(32, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.6))

        # LSTM
        model.add(LSTM(256, return_sequences=False))
        model.add(Dropout(0.5))
        
    if (Architecture == 5):

        # Conv
        model.add(Conv1D(32, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.5))

        # LSTM
        model.add(LSTM(128, return_sequences=False))
        model.add(Dropout(0.5))
    
        model.add(Dense(64,activation = 'relu'))
        
    if (Architecture == 6):

        # Conv
        model.add(Conv1D(256, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.5))

        # LSTM
        model.add(GRU(32, return_sequences=False))
        model.add(Dropout(0.5))
    
        model.add(Dense(64,activation = 'relu'))  
        
        
    # Output Layer
    model.add(Dense(num_classes,activation = 'softmax'))

    # try using different optimizers and different optimizer configs
    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    #optimizer = keras.optimizers.SGD(lr=1,decay=1e-6)
    #optimizer = keras.optimizers.Adagrad(lr=0.001)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])

    print('Train...')

    loss_train = []
    acc_train = []

    loss_val = []
    acc_val = []

    loss_test = []
    acc_test = []


    for i in np.arange(1,N_mont+1):
        print('iteration',str(i)+'/'+str(N_mont))
        
        LIST=np.arange(x_train.shape[0])
        Signal= np.arange(x_train.shape[1])
        
        np.random.shuffle(LIST)
        #np.random.shuffle(Signal)
        
        y_train=y_train[LIST]
        
        x_train =  x_train[LIST] 
        #x_train1 = np.swapaxes(x_train1,0,1) 
        #x_train1 =  x_train1[Signal]
        #x_train1 = np.swapaxes(x_train1,0,1)  
        
        #x_test1 = np.swapaxes(x_test,0,1) 
        #x_test1 =  x_test1[Signal]
        #x_test1 = np.swapaxes(x_test1,0,1)  
        
        history = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1,
                 validation_split = 0.1)

        loss_train.append(np.mean(history.history['loss']))
        acc_train.append(np.mean(history.history['acc']))

        loss_val.append(np.mean(history.history['val_loss']))
        acc_val.append(np.mean(history.history['val_acc']))

        score = model.evaluate(x_test, y_test, verbose=0)

        loss_test.append(score[0])
        acc_test.append(score[1])

        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

# Training for one subject / Test for each and all

In [ ]:
data_table_training = {}
data_table_test = {}
data_table_val = {}

if (Process_data_each_subject == True):
    K.clear_session()    # clearing previous session
    
    num_classes = 4
    signal_length = 1000//Time_devide
    number_channel = 22

    input_shape = (signal_length,number_channel)  

    batch_size = 200//batch_num

    print('Loading data...')
    
    x_train = Train_X[Subject_train]             # 13344 * 125 * 22
    y_train = Train_y[Subject_train]            # 13344

    x_train = x_train.astype('float32')

    # # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)


    print('Build model...')

    model = Sequential()

    if (Architecture == 1):

        # Conv
        model.add(Conv1D(32, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.6))

        # LSTM
        model.add(LSTM(128, return_sequences=False))
        model.add(Dropout(0.5))
        
    if (Architecture == 2):
        
        model.add(Conv1D(32, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.5))
        
        model.add(Conv1D(32, kernel_size = 11))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.5))
        
        model.add(Conv1D(32, kernel_size = 11))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.5))
        
        model.add(Flatten())
        # FC
        model.add(Dense(64,activation = 'relu'))
    
    if (Architecture == 3):

        # Conv
        model.add(Conv1D(64, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.6))
        
        model.add(Conv1D(64, kernel_size = 11))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.6))

        model.add(Flatten())
        # FC
        model.add(Dense(64,activation = 'relu'))
        
    if (Architecture == 4):

        # Conv
        model.add(Conv1D(32, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.6))

        # LSTM
        model.add(LSTM(256, return_sequences=False))
        model.add(Dropout(0.5))
        
    if (Architecture == 5):

        # Conv
        model.add(Conv1D(32, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.5))

        # LSTM
        model.add(LSTM(128, return_sequences=False))
        model.add(Dropout(0.5))
    
        model.add(Dense(64,activation = 'relu'))
        
    if (Architecture == 6):

        # Conv
        model.add(Conv1D(256, kernel_size = 11, input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size=2, strides = 2))
        model.add(Dropout(0.5))

        # LSTM
        model.add(GRU(32, return_sequences=False))
        model.add(Dropout(0.5))
    
        model.add(Dense(64,activation = 'relu'))  


    # # Output Layer
    model.add(Dense(num_classes,activation = 'softmax'))

    # # try using different optimizers and different optimizer configs
    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#     optimizer = keras.optimizers.SGD(lr=1,decay=1e-6)
    #optimizer = keras.optimizers.Adagrad(lr=0.001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    print('Train...')

    loss_train = []
    acc_train = []

    loss_val = []
    acc_val = []

    loss_test = []
    acc_test = []


    for i_test in np.arange(1,num_subject+1):
        
        print('Training Subject: ', Subject_train , 'Testing Subject: ', i_test)
        x_test = Test_X[i_test]                 # 3600, 125, 22
        y_test = Test_y[i_test]                 # 3600
        
        y_test = keras.utils.to_categorical(y_test, num_classes)
        x_test = x_test.astype('float32')
        
        for i in np.arange(1,N_mont):
            LIST=np.arange(x_train.shape[0])
            np.random.shuffle(LIST)
            x_train=x_train[LIST]
            y_train=y_train[LIST]
            
            history = model.fit(x_train, y_train,
                     batch_size=batch_size,
                     epochs=epochs,
                     verbose=0,
                     validation_split = 0.1)

            loss_train.append([Subject_train,i_test,np.mean(history.history['loss'])])
            acc_train.append([Subject_train,i_test,np.mean(history.history['acc'])])
            loss_val.append([Subject_train,i_test,np.mean(history.history['val_loss'])])
            acc_val.append([Subject_train,i_test,np.mean(history.history['val_acc'])])

            score = model.evaluate(x_test, y_test, verbose=0)

            loss_test.append([Subject_train,i_test,score[0]])
            acc_test.append([Subject_train,i_test,score[1]])

In [ ]:
if (Process_data_each_subject == True):
    loss_val_arr=np.array(loss_val)
    loss_val_avg=[]
    m=0
    for ii in np.arange(1,num_subject+1):
        avg=(np.mean(loss_val_arr[0+m:N_mont-1+m],axis=0)[2])
        loss_val_avg.append(avg)
        m=m+2

In [ ]:
if (Process_data_each_subject == True):
    acc_test_arr=np.array(acc_test)
    acc_test_avg=[]
    m=0
    avg=0
    for ii in np.arange(1,num_subject+1):
        avg=(np.max(acc_test_arr[0+m:N_mont-1+m],axis=0)[2])
        acc_test_avg.append(avg)
        m=m+2


    print(acc_test_avg)
    print(np.mean(acc_test_avg))
    print(np.max(acc_test_avg))
    print(np.min(acc_test_avg))

In [ ]:
if (Process_data_each_subject == True):
    acc_test_arr=np.array(acc_test)
    acc_train_arr=np.array(acc_train)
    acc_val_arr=np.array(acc_val)
    acc_test_avg=[]
    acc_train_avg=[]
    acc_val_avg=[]
    m=0
    avg=0
    for ii in np.arange(1,num_subject+1):
        avg_test=(np.max(acc_test_arr[0+m:N_mont-1+m],axis=0)[2])
        avg_train=(np.max(acc_train_arr[0+m:N_mont-1+m],axis=0)[2])
        avg_val=(np.max(acc_val_arr[0+m:N_mont-1+m],axis=0)[2])
        acc_test_avg.append(avg_test)
        acc_val_avg.append(avg_val)
        acc_train_avg.append(avg_train)
        m=m+2

In [ ]:
if (Process_data_each_subject == True):
    print(acc_train_avg)
    print(acc_test_avg)
    print(acc_val_avg)

In [ ]:
if (Process_data_each_subject == True):
    min_acc_train = np.min(acc_train_avg)
    max_acc_train = np.max(acc_train_avg)
    mean_acc_train = np.mean(acc_train_avg)

    min_acc_test = np.min(acc_test_avg)
    max_acc_test = np.max(acc_test_avg)
    mean_acc_test = np.mean(acc_test_avg)

    min_acc_val = np.min(acc_val_avg)
    max_acc_val = np.max(acc_val_avg)
    mean_acc_val = np.mean(acc_val_avg)

    data_table_training[str(Subject_train)] = [min_acc_train,max_acc_train,mean_acc_train]
    data_table_test[str(Subject_train)] = [min_acc_test,max_acc_test,mean_acc_test] 
    data_table_val[str(Subject_train)] = [min_acc_val,max_acc_val,mean_acc_val]

    print('Training across subject:', Subject_train)
    print('Min acc:', min_acc_train,'Max acc:',max_acc_train, 'Mean acc:', mean_acc_train)